In [1]:
TENSORFLOW = 1
GLUON = 1

import os  
if GLUON:
    os.environ['CUDA_VISIBLE_DEVICES']='1'

if TENSORFLOW:
    import tensorflow as tf

    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.set_visible_devices(gpus[0], 'GPU')
            tf.config.set_logical_device_configuration(
                gpus[0],
                [tf.config.LogicalDeviceConfiguration(memory_limit=24000)])
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)


2024-04-09 12:46:10.738007: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 12:46:10.782976: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 12:46:11.573138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1 Physical GPUs, 1 Logical GPUs


2024-04-09 12:46:13.384986: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 24000 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:5e:00.0, compute capability: 7.5


In [2]:
import warnings, pandas as pd, requests, mlflow, sys, os, logging, numpy as np
from automlwrapper import AutoMLWrapper, SedarDataLoader
from sedarapi import SedarAPI

logging.getLogger('automlwrapper').setLevel(logging.ERROR)

np.random.seed(1313)

TIME_LIMIT = 60 * 60

Using TensorFlow backend
WARNING AutoSklearn could not be mported. It might not b available in this environment. Err: 
 No module named 'autosklearn'.
WARNING AutoSklearn could not be imported. It might not be available in this environment. Err: 
 No module named 'autosklearn'.


In [3]:
USE_MLFLOW = False

MLFLOW_URI = 'http://192.168.220.107:6798'
MLFLOW_EXPID = '1'
if USE_MLFLOW:
    mlflow.set_tracking_uri(MLFLOW_URI) 
    mlflow.set_experiment(experiment_id=MLFLOW_EXPID)

In [4]:
SEDAR_URI = 'http://192.168.220.107:5000'

SEDAR = SedarAPI(SEDAR_URI)
DataLoader = SedarDataLoader(SEDAR)

In [5]:
train_location = './tmp/data/coco_train_balanced'
test_location = './tmp/data/coco_test_balanced'

## download data to the above locations

In [6]:
!curl --create-dirs -O --output-dir \
./tmp/data/coco_train_balanced \
https://gitlab.com/mibbels/automlwrapperdata/-/raw/main/object-detection/obj_dect_train_balanced.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5636k  100 5636k    0     0  11.0M      0 --:--:-- --:--:-- --:--:-- 11.0M


In [7]:
!curl --create-dirs -O --output-dir \
./tmp/data/coco_test_balanced \
https://gitlab.com/mibbels/automlwrapperdata/-/raw/main/object-detection/obj_dect_test_balanced.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1412k  100 1412k    0     0  3746k      0 --:--:-- --:--:-- --:--:-- 3757k


## Read the object detection dataset from files
#### Same code utilizing the SEDAR API:
```python
DataLoader.query_data('<workspace id>',
                      '<dataset id>',
                      file_save_location = test_location)
```

In [8]:
test = DataLoader.cocoAsDataFrame(zip_path=test_location, unzip_path=test_location+'/unzip')
train = DataLoader.cocoAsDataFrame(train_location, train_location + '/unzip')

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
train.describe()

,image,rois,label
count,140,140,140
unique,140,140,140
top,/home/jovyan/tmp/data/coco_train_balanced/unzi...,"[[462.999552, 237.999744, 703.0, 461.999935999...","[[462.999552, 237.999744, 703.0, 461.999935999..."
freq,1,1,1


In [10]:
test.describe()

,image,rois,label
count,36,36,36
unique,36,36,36
top,/home/jovyan/tmp/data/coco_test_balanced/unzip...,"[[379.00032, 336.99993600000005, 586.00032, 52...","[[379.00032, 336.99993600000005, 586.00032, 52..."
freq,1,1,1


# Optimizing a single predictor using AutoGluon

In [ ]:
def wrapper_medium(train_data, eval_metric):
    wrapper = AutoMLWrapper('autogluon')
    
    wrapper.Train(
        train_data=train_data,
        target_column='',
        task_type='object-detection',
        data_type='image',
        problem_type='object-detection',
        hyperparameters={'time_limit': TIME_LIMIT, 
                         'preset' : 'best_quality',
                        'eval_metric' : eval_metric,
                         'validation_metric': eval_metric,
                        'verbosity':1, 
                        }
    )
    
    return wrapper

In [ ]:
%%time
w = wrapper_medium(train,'map')

In [ ]:
%%time

test_map = w.Evaluate(test_data=test, target_column='label')
print(test_map)

-------------

# Evaluating the LLMs capabilities when creating object detection code

## **Trial 1**: An earlier iteration of the prompt, relying on the runtime to add information on the data (coco annotation sample) and added image information from SEDAR 
-  shorter prompts turned out to be preferred
-  'copy and paste' of a serialization of the Neo4j metadata in SEDAR (seen after "information sample image:") provides the metadata in a convoluted structure

In [ ]:
YOUR_MESSAGE = 'locate the correct type of coating-defect in the image'


print(f'''Your task is to create the Python code for a Machine Learning Pipeline. You will do that by outputting comment-annotated Python code
using the libraries [scikit-learn, Tensorflow, Pytorch].
GPUs are available in this environment.

Here are some more details on the type of problem the ML Pipeline should solve:                                
data_type=image
task_type=object-detection
problem_type=object-detection
The target-variable is: label

The Dataset in question is called coco_22_train_v1. 



        Here is some metadata for the dataset.


All data is contained in the dataframe coco_22_train_v1.
'rois' and 'label' are the same column. They contain a list of lists with:
[[xmin, ymin, xmax, ymax, class], [...]]

coco_22_train_v1.iloc[0]:

{train.iloc[0]}


        information sample image:  <"id": "9b673c7013234b6ea394909bc5a34230", "pathFull": "coco_22_balanced/images/stippen_blau_2_10_Height.jpg", "imageHeight": "768", "imageWidth": "1024", "bitsPerPixel": "8", "colorMode": "GREY", "format": "jpg">

        folder-structure of the dataset zip-file:  <"uid": "e604e262a23d452b8e9145f06595e981", "name": "coco_22_balanced", "type": "ROOT", "subfolders": [ <"uid": "378e974a2348413aa2ee4b2af10474f9", "name": "annotations", "type": "COCO_ANNOTATION", "subfolders": []>,  <"uid": "5fc438ee133c422d8c325174050f5e89", "name": "images", "type": "COCO_IMAGE", "subfolders": []>]>

The user was prompted to provide further descriptions. If the text was provided, it will follow now:
 {YOUR_MESSAGE}                         
''')

#   The above prompt was given to the OS model 'Hermes'
Answer 1:

I understand that you want me to help you create a Machine Learning Pipeline for object detection using Tensorflow or Pytorch on GPUs with coco_22_train_v1 dataset. The target variable is label and the task type is object detection. Is this correct?

Prompt 2:

Yes, please give me the code to execute this task.

Answer 2:
```python
import tensorflow as tf
from tensorflow import keras
# Load the dataset into a pandas dataframe.
coco_df = pd.read_csv('path/to/dataset.csv')
Extract the rois and labels from the dataframe.
rois = coco_df['rois'].tolist()
labels = coco_df['label'].tolist()
Convert the rois to a numpy array of shape (num_rois, 5).
num_rois = len(rois)
rois_np = np.array(rois)
Reshape the rois and labels into tensors for input to the model.
rois_tensor = tf.reshape(rois_np, (num_rois, 5))
labels_tensor = tf.constant(labels)
Define a custom object detection model using Tensorflow.
model = keras.Sequential([
    keras.layers.Conv2D(32
```
Prompt 3:

Can you finish this code to complete the construction of the neural network?

Answer 3:

 Sure, here's an example of how you can define the rest of the layers in the model and train it on the dataset using Tensorflow on GPUs:

```python


    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(.5),
    keras.layers.Dense(num_classes, activation='softmax')],
name='detection_model',
weights=[tf.keras.preprocessing.image.load_img_class_names('path/to/coco_labels.txt'),
         tf.keras.applications.mobilenet_v2.decode_predictions])(inputs)
# Compile the model with appropriate loss function, optimizer and metrics.
model.compile(optimizer='adam',
              loss=tf.keras.losses.Categor

```


 # **Trial 2**: Below is a custom prompt which tries to convey the task more clearly


Your task is to create the Python code for a Machine Learning Pipeline. You will do that by outputting comment-annotated Python code using the libraries [scikit-learn, Tensorflow, Pytorch]. GPUs are available in this environment.

The code should solve an objkect detection problem with a dataset in COCO format.

The COCO file is located at: tmp/data/coco_script_trainv1/unzip/coco_script_trainv1/annotations/labels.json There are 314 images.

This are the category names: "categories": [ { "id": 0, "name": "kocher" }, { "id": 1, "name": "krater" }, { "id": 2, "name": "stippen" }, { "id": 3, "name": "blasen" } ]

Here is information about the format of each image: imageHeight: 768px, imageWidth: 1024px, colorMode: greyscale format: jpg

Create the python code for object detection with a deep neural network.


# Hermes's code was again cut short and we needed to re-prompt

 Sure! Here's an example code that uses Tensorflow to train a Faster R-CNN model on COCO dataset:
```python
# Importing necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# Loading the COCO dataset
import cocoapi.CocoDetection
detections = cocoapi.COCO(tmp/data/coco_script_trainv1/unzip/coco_script_trainv1/annotations/instances_val2017.json)
# Splitting the dataset into training and validation sets
train_idx, val_idx = train_test_split(detections.ids, test_size=0.2, random_state=42)
Training the Faster R-RCNN model using Tensorflow
model = keras.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(76
```

Prompt 2:

Continue giving me the Code for building and training the Faster R-CNN model using Tensorflow

Answer 2:
Sure! Here's the rest of the code:
```python
, 1024)),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Conv2D(64, (3, 3), activation='relu'),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Flatten(), 
layers.Dense(1024, activation='relu'), 
layers.Dropout(.5), 
layers.Dense(314) ])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.DiceLoss()], ) 
model.fit(train_dataset, epochs=5)
Evaluating the model on validation set

val_score = model.evaluate(val_dataset, verbose=0) print('\nFinal evaluation results:') print('Test loss:', val_score[0
```


#### The provided code does not try to load the data from the DataFrame, instead it relies on a label file using a made-up library
#### The layers as well as metric and loss are not fit for object detection  

# **Trial 3**: This next prompt relies on the COCO annotation file 

Generate a Python code snippet for an object detection pipeline using either TensorFlow or PyTorch, optimized for GPU usage. The code should handle a COCO-format dataset located at "tmp/data/coco_script_trainv1/unzip/coco_script_trainv1/annotations/labels.json". The dataset contains 314 images. All images are 768px in height, 1024px in width, in greyscale, and saved as JPGs. Focus on a simple yet effective neural network architecture suitable for beginners. Include necessary imports, dataset loading and preprocessing steps, model architecture, and training loop. The code should be ready to run in a Python interpreter without modifications Annotate the code with comments to explain each step.



## Here we used OpenOrca as the OS LLM
Here's an example of a simple yet effective neural network architecture for object detection using TensorFlow on GPU:
```python
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_file

# Download the COCO dataset:
coco_dataset = "tmp/data/coco_script_trainv1/unzip/coco_script_trainv1/annotations/labels.json"

Preprocess the images and labels for training:
def preprocess(img, label):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    return tf.expand_dims(img_array, 0), label

Load the dataset using ImageDataGenerator:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory="tmp/data/coco_script_trainv1/unzip/coco_script
```

Prompt 2:

Continue to write this code for loading a coco dataset using ImageDataGenerator.

Answer 2:
```python
 trainv1/annotations", x_col='filename', y_col='labels') class_mode="sparse")
```

Prompt 3:

Build the neural network architecture

Answer 3:
```python
model = keras.Sequential([ layers.Conv2D(32, (3, 3), activation='relu'),
                          layers.MaxPooling2D((2, 2)),
                          layers.Conv2D(64, (3, 3), activation='relu'),
                          layers.MaxPooling2D((2, 2)),
                          layers.Flatten(),
                          layers.Dense(1024, activation='relu'),
                          layers.Dense(len(class_names), activation='softmax')
                         ])

#Compile the model and train it:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_generator, epochs=10)


#This code snippet demonstrates a simple yet effective neural network architecture for object detection using TensorFlow on GPU. It loads the COCO dataset from the specified path
```

#### It becomes evident that the task should be split across multiple prompts, each addressing specific problems like data-loading, model creation, etc. 

# **Trial 4**:This custom prompt tackels only data loading from the coco annotatios file

"tmp/data/coco_script_trainv1/unzip/coco_script_trainv1/annotations/labels.json" contains the annotation file for an object detection dataset in coco format. Provide the code to load this dataset and turn it into a format compatible with tensorflow. 

## Orca seems to fail without a more detailed description on the structure of the file

```python
import json
from PIL import Image
import numpy as np
import tensorflow as tf

def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

def convert_to_tfrecord(data):
    for image_id, annotations in data.items():        
        print(image_id)
        print(annotations)
        width, height = Image.open(image_id).size
        xmin, ymin, xmax, ymax = [int(float(x)) for x in annotations['bbox']]
        class_name = annotations['category']['name']
        label = int(annotations['category']['id'])
        tfrecord = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset.tensor_info(width),
            'image/width': dataset.tensor_info(height),
            'image/filename': dataset.string_list_feature([image_id]),
        'image/object/bbox/xmin': dataset.float_list_feature([xmin]),
        'image/object/bbox/ymin': dataset.float_list_feature([ymin]),
        'image/object/bbox/xmax': dataset.float_list_feature([xmax]),
        'image/object/bbox/ymax': dataset.float_list_feature([ymax]),
        'image/object/class/label': dataset.int64_list_feature([label]),
        'image/object/class/text': dataset.string_list_feature([class_name])
    }))
    serialized = tfrecord.SerializeToString()
    with tf.io.TFRecordWriter(filepath) as writer:
        writer.write(serialized)

coco_dataset = "tmp/data/coco_script_trainv1/unzip/coco_script_trainv1/annotations/labels.json"

data = load_json(coco_dataset)

convert_to_tfrecord(data)
```

# **Trial 5**: Final fully functional pipeline with GPT4, achieved by prompting each step seperatly


In [11]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from pycocotools.coco import COCO
from torchvision import transforms

class CocoDetection(Dataset):
    def __init__(self, root, annotation, transform=None):
        self.root = root
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.transform = transform

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        coco_annotation = coco.loadAnns(ann_ids)
        path = coco.loadImgs(img_id)[0]['file_name']

        img = Image.open(os.path.join(self.root, path)).convert('L')  # Convert to grayscale
        
        boxes = [ann['bbox'] for ann in coco_annotation]
        categories = [ann['category_id'] for ann in coco_annotation]          # EDITED   Added (labels was wrongly torch.ones)
        categories = torch.as_tensor(categories, dtype=torch.int64)
        
        # Convert boxes (x, y, width, height) to (x_min, y_min, x_max, y_max)
        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)
        boxes[:, 2:] += boxes[:, :2]

        # Assuming all instances are not crowd
        #labels = torch.ones((len(boxes),), dtype=torch.int64)     #EDITED  wrong!
        
        target = {}
        target['boxes'] = boxes
        #target['labels'] = labels
        target['labels'] = categories                        #EDITED 

        if self.transform:
            img = self.transform(img)
                
        return img, target, img_id

    def __len__(self):
        return len(self.ids)


In [12]:

def create_data(train_path, test_path):
    from sklearn.model_selection import train_test_split
    from torch.utils.data._utils.collate import default_collate
    
    transform = transforms.Compose([
        transforms.ToTensor(),  # Converts PIL.Image or np.ndarray to FloatTensor of shape (C x H x W) and normalize to [0.0, 1.0]
        # Add any other transformations here (e.g., resizing, normalization)
    ])
    
    def collate_fn(batch):   # Result of question for RuntimeError in Evaluate Loop 
        """
        Custom collate function for handling None or varying sizes of bounding box annotations.
        """
        batch = list(filter(lambda x: x is not None, batch))  # Remove None samples if any
        images = [item[0] for item in batch]
        targets = [item[1] for item in batch]
        
        images = default_collate(images)  # Use PyTorch's default_collate to handle images
        
        # For targets, simply return the list without attempting to stack
        
        # You might need to handle targets differently based on your specific needs
        return images, targets
    
    
    
    train_full = CocoDetection(root=train_path, annotation=train_path+'annotations/labels.json', transform=transform)
    
    indices = list(range(len(train_full)))
    
    # Split indices into training and validation sets
    train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)
    
    # Create subset instances for training and validation
    train_dataset = torch.utils.data.Subset(train_full, train_indices)
    val_dataset = torch.utils.data.Subset(train_full, val_indices)
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  collate_fn=lambda x: tuple(zip(*x)))
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    
    test_dataset = CocoDetection(root=test_path, annotation=test_path + 'annotations/labels.json', transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))#collate_fn=collate_fn)

    return train_loader, val_loader, test_loader, test_dataset

In [13]:
def model_train(device, data_loader, num_epochs = 50, pretrained=True):
    from torchvision.models.detection import fasterrcnn_resnet50_fpn
    import torchvision
    
    def get_model_instance_segmentation(num_classes):
        model = fasterrcnn_resnet50_fpn(pretrained)       #EDITED added variable
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
        return model
    
    model = get_model_instance_segmentation(num_classes=5) # for example   ### HAS TO BE EDITED!!  
    
    
    model.to(device)
    
    # Optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    
    
    # Training loop
    
    for epoch in range(num_epochs):
        model.train()
        i = 0
        for images, targets, image_ids  in data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
    
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            if i % 10 == 0:
                print(f"Iteration #{i} loss: {losses.item()}")
            i += 1

    return model

In [14]:
def model_eval(device, model, data_loader, dataset):
    model.eval()  # Set the model to evaluation mode
    import json
    
    results = []
    
    with torch.no_grad():
        for images, targets, image_ids in data_loader:
            images = [image.to(device) for image in images]
            outputs = model(images)
    
            for i, output in enumerate(outputs):
                image_id = image_ids[i]#.item()  # Assuming image_ids is a tensor, adjust if it's in another format
                for box, label, score in zip(output['boxes'], output['labels'], output['scores']):
                    # Convert box from [x1, y1, x2, y2] to [x, y, width, height]
                    box = box.cpu().numpy()
                    x_min, y_min, x_max, y_max = box
                    width, height = x_max - x_min, y_max - y_min
                    
                    result = {                      # EDITED:: float() added, error encoding np.float32 
                        "image_id": image_id,
                        "category_id": label.item(),
                        "bbox": [float(x_min), float(y_min), float(width), float(height)],
                        "score": float(score.item())
                    }
                    results.append(result)
    
    
    with open('predictions.json', 'w') as f:
        json.dump(results, f)


In [15]:
def coco_eval(path):

    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    
    cocoGt = COCO(path + '/annotations/labels.json')  # Load the ground truth
    cocoDt = cocoGt.loadRes('predictions.json')  # Load the predictions
    
    cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')  # Initialize COCOeval for bounding box evaluation
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()



In [16]:
%%time
train_path = './tmp/data/coco_train_balanced/unzip/obj_dect_train_balanced/'
test_path = './tmp/data/coco_test_balanced/unzip/obj_dect_test_balanced/'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

train, val, test, test_ds = create_data(train_path, test_path)
model = model_train(device, train, 40)
model_eval(device, model, test, test_ds)
coco_eval(test_path)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


/usr/local/automl/lib/python3.11/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/automl/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Iteration #0 loss: 3.95310378074646
Iteration #0 loss: 1.1223046779632568
Iteration #0 loss: 0.8583314418792725
Iteration #0 loss: 0.5336757898330688
Iteration #0 loss: 0.4925512671470642
Iteration #0 loss: 0.43428030610084534
Iteration #0 loss: 0.7340050339698792
Iteration #0 loss: 0.48963961005210876
Iteration #0 loss: 0.3516971468925476
Iteration #0 loss: 0.6191205382347107
Iteration #0 loss: 0.38004398345947266
Iteration #0 loss: 0.38578230142593384
Iteration #0 loss: 0.3294800817966461
Iteration #0 loss: 0.3977416753768921
Iteration #0 loss: 0.3722078204154968
Iteration #0 loss: 0.37976548075675964
Iteration #0 loss: 0.3547268509864807
Iteration #0 loss: 0.3614848554134369
Iteration #0 loss: 0.4079762399196625
Iteration #0 loss: 0.46287861466407776
Iteration #0 loss: 0.34341728687286377
Iteration #0 loss: 0.3050822913646698
Iteration #0 loss: 0.20184817910194397
Iteration #0 loss: 0.20880062878131866
Iteration #0 loss: 0.254375159740448
Iteration #0 loss: 0.35746312141418457
Itera